---

# 🥉 LEVEL 1: Access Control Designer

**Difficulty:** Intermediate | **Time:** 30 minutes

## 📚 What You'll Learn

- Unity Catalog permission model
- GRANT and REVOKE syntax
- Difference between SELECT, MODIFY, CREATE
- Schema-level vs table-level permissions

## 🎯 Your Mission

Design the basic access control model for the three teams. No fancy features yet - just get the permissions right.

## 📝 Task Instructions

### Step 1: Understand the Current Tables

First, explore what tables exist:

In [ ]:
-- Check what tables we have from the pipeline
%sql
SHOW TABLES IN main.e2eai_iot_turbine;

### Step 2: Design Schema Architecture

**Question:** Should all teams access the same schema, or create separate schemas?

**Option A - Single Schema:**
```
main.e2eai_iot_turbine
  ├─ sensor_bronze (all teams READ)
  ├─ sensor_hourly (all teams READ)
  ├─ turbine_status (all teams READ)
  └─ turbine_maintenance (field techs only)
```

**Option B - Multiple Schemas (Recommended):**
```
main.turbine_operations (Team A - Field Techs)
  ├─ sensor_readings_view
  ├─ maintenance_schedule
  └─ turbine_status
  
main.turbine_analytics (Team B - Data Scientists)
  ├─ all_sensor_data
  ├─ ml_features
  └─ experiment_sandbox (WRITE access here)
  
main.turbine_executive (Team C - Execs)
  ├─ fleet_kpis_view
  ├─ cost_analysis_view
  └─ performance_summary
```

**Your Task:** Choose your architecture and document it below.

### 📝 Your Schema Design

Document your decision:

```
My Architecture Choice: [Option A / Option B / Custom]

Reasoning:
- [Why did you choose this structure?]
- [How does it meet security requirements?]
- [What are the tradeoffs?]

Schema Structure:
Schema 1: _______________
  Purpose: _______________
  Teams with access: _______________
  
Schema 2: _______________
  Purpose: _______________
  Teams with access: _______________

[Add more schemas if needed]
```

### Step 3: Create User Groups

In Unity Catalog, you manage permissions via **groups**, not individual users.

**Task:** Create groups for each team (or use existing ones):

In [ ]:
-- Create groups for each team
-- Note: You may need admin rights to create groups
-- In Free Edition, groups might be pre-created

-- Check existing groups first
%sql
SHOW GROUPS;

In [ ]:
-- Create new groups (if needed)
-- Uncomment if you have permissions:

-- CREATE GROUP IF NOT EXISTS field_technicians;
-- CREATE GROUP IF NOT EXISTS data_scientists;
-- CREATE GROUP IF NOT EXISTS executives;

-- Verify groups exist
%sql
SHOW GROUPS;

### Step 4: Grant Permissions - Team A (Field Technicians)

**Requirements:**
- READ access to operational data
- NO access to financial tables
- NO WRITE access (they shouldn't modify data)

**Implementation:**

In [ ]:
-- GRANT statements for Team A - Field Technicians

-- Option 1: Grant at schema level (easiest)
-- GRANT USE SCHEMA ON SCHEMA main.e2eai_iot_turbine TO `field_technicians`;
-- GRANT SELECT ON SCHEMA main.e2eai_iot_turbine TO `field_technicians`;

-- Option 2: Grant at table level (more granular control)
-- Example:
GRANT SELECT ON TABLE main.e2eai_iot_turbine.sensor_hourly TO `field_technicians`;
GRANT SELECT ON TABLE main.e2eai_iot_turbine.turbine_status TO `field_technicians`;
GRANT SELECT ON TABLE main.e2eai_iot_turbine.turbine_maintenance TO `field_technicians`;

-- Add more GRANT statements based on your architecture
-- YOUR CODE HERE

### Step 5: Grant Permissions - Team B (Data Scientists)

**Requirements:**
- READ access to ALL sensor data (for ML training)
- WRITE access to analytics/experiment schema
- NO access to production tables for writing

**Implementation:**

In [ ]:
-- GRANT statements for Team B - Data Scientists

-- Read access to all tables
GRANT SELECT ON SCHEMA main.e2eai_iot_turbine TO `data_scientists`;

-- Write access to dedicated analytics schema (if you created one)
-- GRANT ALL PRIVILEGES ON SCHEMA main.turbine_analytics TO `data_scientists`;

-- Or specific table-level grants:
-- GRANT SELECT, MODIFY ON TABLE main.turbine_analytics.ml_features TO `data_scientists`;

-- YOUR CODE HERE

### Step 6: Grant Permissions - Team C (Executives)

**Requirements:**
- READ access ONLY to aggregated views/dashboards
- NO access to raw sensor data
- NO access to technical tables

**Strategy:** Create views first, then grant access to views only.

In [ ]:
-- First, create an executive KPI view
CREATE OR REPLACE VIEW main.e2eai_iot_turbine.executive_kpi_dashboard AS
SELECT
  DATE(hour_window.start) as date,
  COUNT(DISTINCT turbine_id) as active_turbines,
  SUM(avg_power) as total_power_output_kw,
  AVG(avg_power) as avg_power_per_turbine
  -- Add more KPIs but NO raw sensor data!
FROM main.e2eai_iot_turbine.sensor_hourly
GROUP BY DATE(hour_window.start);

In [ ]:
-- GRANT access to view only (NOT underlying tables)
GRANT SELECT ON VIEW main.e2eai_iot_turbine.executive_kpi_dashboard TO `executives`;

-- Executives should NOT have access to raw tables:
-- REVOKE SELECT ON TABLE main.e2eai_iot_turbine.sensor_bronze FROM `executives`;

-- YOUR CODE HERE

### Step 7: Verify Permissions

Check what permissions you've granted:

In [ ]:
-- Check grants for a specific table
%sql
SHOW GRANTS ON TABLE main.e2eai_iot_turbine.sensor_hourly;

In [ ]:
-- Check grants for a schema
%sql
SHOW GRANTS ON SCHEMA main.e2eai_iot_turbine;

## ✅ Validation Steps

1. **Test as different users** (if possible):
   - Log in as field_technician → Can you read sensor_hourly? ✓
   - Try to modify data → Should FAIL ✗
   
2. **Check permissions hierarchy:**
   ```sql
   -- Schema-level permissions should apply to all tables
   -- Table-level permissions override schema-level
   ```

3. **Document your grants:**
   - Which team can access which tables?
   - What permission level? (SELECT, MODIFY, ALL)

## 📊 Success Criteria

✅ Created user groups for all 3 teams  
✅ Granted appropriate READ permissions  
✅ Restricted WRITE access to data scientists only  
✅ Created executive view with aggregated data only  
✅ Verified permissions with SHOW GRANTS  

**Level 1 Complete!** 🎉 You've designed basic access control!

---

# 🥈 LEVEL 2: Security Architect

**Difficulty:** Advanced | **Time:** 45 minutes

## 📚 What You'll Learn

- Data masking techniques
- Row-level security (filtering)
- Column-level access control
- Handling edge cases (users in multiple groups)

## 🎯 Your Mission

The basic permissions work, but there are problems:
1. Field techs can see turbine purchase costs (confidential!)
2. Execs accidentally saw raw sensor errors (confusing!)
3. A user in multiple groups has wrong permissions

**Goal:** Implement advanced security features to handle these issues.

## 📝 Task Instructions

### Step 1: Identify Sensitive Columns

**Review your tables and identify sensitive data:**

```
Table: turbine_maintenance
- Sensitive columns: cost_estimate, vendor_pricing
- Who should see: Executives only

Table: sensor_hourly
- Sensitive columns: None (but should be aggregated for execs)

Table: turbine_status
- Sensitive columns: failure_reason_internal (technical details)
- Who should see: Data scientists only
```

**Your Task:** List sensitive columns in your tables.

### 📝 Your Sensitive Data Audit

Document what needs protection:

```
Table: _______________________
Column: _______________________
  Sensitivity: [Confidential / Internal / Public]
  Accessible by: [Which teams?]
  Masking strategy: [Redact / Hash / Aggregate / Filter]

[Repeat for all sensitive columns]
```

### Step 2: Implement Data Masking

Unity Catalog supports **column masking** using SQL functions.

**Strategy:** Create masked views for teams that shouldn't see raw data.

In [ ]:
-- Create a masked view for Field Technicians
-- Hide cost information from them

CREATE OR REPLACE VIEW main.e2eai_iot_turbine.sensor_hourly_field_view AS
SELECT
  turbine_id,
  window,
  avg_power,
  avg_vibration,
  avg_temperature,
  -- Mask any cost-related fields
  '***REDACTED***' as cost_per_hour  -- If this column exists
FROM main.e2eai_iot_turbine.sensor_hourly;

In [ ]:
-- Now grant field techs access to MASKED VIEW instead of raw table
REVOKE SELECT ON TABLE main.e2eai_iot_turbine.sensor_hourly FROM `field_technicians`;
GRANT SELECT ON VIEW main.e2eai_iot_turbine.sensor_hourly_field_view TO `field_technicians`;

### Step 3: Implement Row-Level Security

**Scenario:** Different field technicians work in different regions. They should only see turbines in THEIR region.

**Implementation:** Create views with row filters based on user attributes.

In [ ]:
-- First, create a user-region mapping table
CREATE OR REPLACE TABLE main.e2eai_iot_turbine.user_regions (
  username STRING,
  allowed_region STRING
);

-- Insert sample data
INSERT INTO main.e2eai_iot_turbine.user_regions VALUES
  ('tech_chicago@company.com', 'Chicago'),
  ('tech_miami@company.com', 'Miami'),
  ('tech_nyc@company.com', 'New York');

In [ ]:
-- Create row-filtered view based on current user
CREATE OR REPLACE VIEW main.e2eai_iot_turbine.sensor_hourly_my_region AS
SELECT s.*
FROM main.e2eai_iot_turbine.sensor_hourly s
-- Join with turbine location info (assuming location column exists)
-- INNER JOIN main.e2eai_iot_turbine.turbine t ON s.turbine_id = t.turbine_id
WHERE 
  -- Filter based on current user's region
  -- t.location IN (
  --   SELECT allowed_region 
  --   FROM main.e2eai_iot_turbine.user_regions 
  --   WHERE username = current_user()
  -- )
  
  -- Simplified version if location is in sensor_hourly:
  1=1  -- Replace with actual row filter logic
;

### Step 4: Column-Level Security

**Advanced:** Different users see different columns in the same table.

**Example:** Data scientists see all columns, field techs see subset.

In [ ]:
-- Create role-specific views with different column sets

-- View for Field Technicians (limited columns)
CREATE OR REPLACE VIEW main.e2eai_iot_turbine.turbine_status_field AS
SELECT
  turbine_id,
  status,  -- operational_status
  last_maintenance_date
  -- EXCLUDE: cost_per_kwh, failure_prediction_score, internal_notes
FROM main.e2eai_iot_turbine.turbine_status;

-- View for Data Scientists (all columns including ML features)
CREATE OR REPLACE VIEW main.e2eai_iot_turbine.turbine_status_analytics AS
SELECT
  *,  -- All columns
  failure_prediction_score,
  ml_feature_vector
FROM main.e2eai_iot_turbine.turbine_status;

In [ ]:
-- Grant appropriate views to each group
GRANT SELECT ON VIEW main.e2eai_iot_turbine.turbine_status_field TO `field_technicians`;
GRANT SELECT ON VIEW main.e2eai_iot_turbine.turbine_status_analytics TO `data_scientists`;

### Step 5: Handle Edge Cases

**🚨 Problem:** Sarah is BOTH a Field Technician AND a Data Scientist.

**Question:** What permissions does she get?
- Most permissive (Data Scientist level)? ✓ Unity Catalog default
- Most restrictive (Field Technician level)? ✗ Not default
- Custom combination? (Advanced)

**Your Task:** Document how you would handle this.

### 📝 Your Edge Case Strategy

**Scenario Analysis:**

**Case 1: User in Multiple Groups**
```
User: sarah@company.com
Groups: field_technicians, data_scientists
Expected behavior: [Most permissive / Most restrictive / Custom]
Your implementation: [How will you configure this?]
```

**Case 2: Temporary Access**
```
A contractor needs 1-week access to specific turbine data.
Your solution: [How do you grant time-limited access?]
```

**Case 3: Emergency Override**
```
During an outage, operations team needs immediate access to ALL data.
Your solution: [Break-glass procedure?]
```

### Step 6: Test Your Security Model

**Validation queries:**

In [ ]:
-- Test 1: Can field tech see masked cost data?
-- (Log in as field_technician or simulate)
SELECT * FROM main.e2eai_iot_turbine.sensor_hourly_field_view LIMIT 5;
-- Expected: cost columns should show '***REDACTED***'

In [ ]:
-- Test 2: Can executive see raw sensor data?
-- (Log in as executive)
-- SELECT * FROM main.e2eai_iot_turbine.sensor_bronze LIMIT 5;
-- Expected: Should FAIL with permission denied

-- Instead, executive should only access:
SELECT * FROM main.e2eai_iot_turbine.executive_kpi_dashboard;
-- Expected: SUCCESS, aggregated data only

In [ ]:
-- Test 3: Check all grants for a user in multiple groups
SHOW GRANTS ON TABLE main.e2eai_iot_turbine.sensor_hourly;
-- Expected: Both groups listed, most permissive wins

## 🎓 Bonus Challenges (Optional)

### Bonus 1: Dynamic Masking Function

Create a SQL function that masks data based on current user:

```sql
CREATE FUNCTION mask_if_not_authorized(value STRING, authorized_group STRING)
RETURNS STRING
RETURN CASE 
  WHEN is_member(authorized_group) THEN value
  ELSE '***'
END;

-- Use in view:
SELECT 
  turbine_id,
  mask_if_not_authorized(cost_estimate, 'executives') as cost
FROM turbine_maintenance;
```

### Bonus 2: Audit Logging

Track who accessed what:

```sql
CREATE TABLE main.e2eai_iot_turbine.access_audit (
  username STRING,
  table_accessed STRING,
  access_time TIMESTAMP,
  query_text STRING
);

-- Use Unity Catalog audit logs to populate this
```

## ✅ Validation Steps

1. **Masking works:** Field techs see '***' for sensitive columns
2. **Row filters work:** Users only see their region's data
3. **Column-level security:** Different roles see different columns
4. **Edge cases handled:** Multi-group users have correct permissions
5. **Tests pass:** All validation queries behave as expected

## 📊 Success Criteria

✅ Implemented data masking for sensitive columns  
✅ Created row-level security filters  
✅ Set up column-level access control  
✅ Documented edge case handling strategy  
✅ Validated security with test queries  

**Level 2 Complete!** 🎉 You're now a security architect!

---

# 🥇 LEVEL 3: Governance Master

**Difficulty:** Expert | **Time:** 60 minutes

## 📚 What You'll Learn

- Complete governance framework design
- Audit logging and compliance
- Data lineage for impact analysis
- Automated access reviews
- Disaster recovery for permissions

## 🎯 Your Mission

The CTO is impressed! Now they want a **complete enterprise governance framework**:

1. Automated compliance reporting
2. Data lineage tracking
3. Regular access reviews
4. Incident response procedures

**Goal:** Build a production-ready governance system that scales.

## 📝 Implementation Guide

### Component 1: Governance Documentation

**Create a governance playbook that includes:**


### 📋 Your Governance Playbook

Complete this template:

---

## **WIND TURBINE DATA GOVERNANCE PLAYBOOK**

### 1. Access Control Matrix

| Role | Schema Access | Table Access | Permission Level | Approval Required |
|------|---------------|--------------|------------------|-------------------|
| Field Technician | turbine_operations | sensor_hourly, turbine_status | SELECT | Manager approval |
| Data Scientist | turbine_analytics | ALL | SELECT, MODIFY | Lead DS approval |
| Executive | turbine_executive | KPI views only | SELECT | CTO approval |
| Admin | ALL | ALL | ALL | Security team |

### 2. Data Classification

| Classification | Examples | Who Can Access | Encryption | Audit Level |
|----------------|----------|----------------|------------|-------------|
| Public | Fleet size, location names | All employees | No | Basic |
| Internal | Sensor readings | Tech + DS + Execs | TLS | Standard |
| Confidential | Cost data, failure predictions | Execs + DS only | TLS + at-rest | Enhanced |
| Restricted | Strategic plans | Execs only | Full encryption | Complete |

### 3. Incident Response Procedures

**Scenario: Unauthorized Access Detected**
1. Step 1: [Immediately revoke access]
2. Step 2: [Check audit logs]
3. Step 3: [Notify security team]
4. Step 4: [Review similar permissions]

**Scenario: Data Leak**
1. Step 1: [___]
2. Step 2: [___]
3. Step 3: [___]

### 4. Access Review Schedule

- **Monthly:** Review new user additions
- **Quarterly:** Audit all field technician access
- **Annually:** Complete access recertification
- **Ad-hoc:** After employee role changes

### 5. Compliance Checklist

- [ ] GDPR: Personal data minimized
- [ ] SOC 2: Access controls documented
- [ ] ISO 27001: Periodic access reviews
- [ ] Internal: Cost data protected
- [ ] Internal: Audit trail enabled

---

### Component 2: Audit Logging System

**Task:** Set up comprehensive audit logging.

In [ ]:
-- Create audit tables

CREATE TABLE IF NOT EXISTS main.governance.access_audit_log (
  event_id STRING,
  event_time TIMESTAMP,
  username STRING,
  user_groups ARRAY<STRING>,
  action STRING,  -- SELECT, INSERT, UPDATE, DELETE, GRANT, REVOKE
  target_catalog STRING,
  target_schema STRING,
  target_table STRING,
  query_text STRING,
  success BOOLEAN,
  error_message STRING
);

-- In production, this would be populated by Unity Catalog system tables
-- Example: Use Databricks audit logs or Unity Catalog audit logs

In [ ]:
-- Create a view to analyze access patterns

CREATE OR REPLACE VIEW main.governance.suspicious_access_patterns AS
SELECT
  username,
  target_table,
  COUNT(*) as access_count,
  COUNT(DISTINCT DATE(event_time)) as days_accessed,
  MAX(event_time) as last_access
FROM main.governance.access_audit_log
WHERE 
  action = 'SELECT'
  AND success = true
GROUP BY username, target_table
HAVING 
  -- Flag unusual patterns
  access_count > 1000  -- More than 1000 queries to same table
  OR days_accessed < 2  -- Accessed only 1-2 days (bulk download?)
ORDER BY access_count DESC;

### Component 3: Data Lineage Tracking

**Purpose:** Understand data flow and impact of changes.

**Unity Catalog provides lineage automatically!** But you should document critical flows:

### 📊 Critical Data Lineage Map

Document your data flows:

```
SOURCE → TRANSFORMATION → DESTINATION
========================================

Flow 1: Sensor Data to Executive Dashboard
sensor_bronze (raw IoT stream)
  → sensor_hourly (hourly aggregation)
  → executive_kpi_dashboard (view)
  → Databricks SQL Dashboard
  
Impact: If sensor_hourly changes, executives see different KPIs!

Flow 2: ML Pipeline
sensor_bronze → sensor_silver_hourly → turbine_gold_anomalies → ML Model

Impact: Changes to sensor_bronze affect model predictions!

[Document your lineage here]
```

**Query lineage in Unity Catalog:**
- Go to Databricks UI → Data Explorer
- Click on table → "Lineage" tab
- See upstream and downstream dependencies

### Component 4: Automated Access Reviews

**Task:** Create queries to support quarterly access reviews.

In [ ]:
-- Query 1: List all users with access to sensitive tables

CREATE OR REPLACE VIEW main.governance.sensitive_table_access AS
SELECT
  grantee as username,
  table_catalog,
  table_schema,
  table_name,
  privilege_type,
  is_grantable
FROM system.information_schema.table_privileges
WHERE 
  table_name IN ('turbine_maintenance', 'cost_analysis', 'turbine_gold_anomalies')
  AND grantee NOT IN ('admin', 'system')
ORDER BY table_name, username;

In [ ]:
-- Query 2: Find users with excessive permissions

CREATE OR REPLACE VIEW main.governance.excessive_permissions_report AS
WITH user_permission_count AS (
  SELECT
    grantee as username,
    COUNT(DISTINCT table_name) as table_access_count,
    COLLECT_SET(privilege_type) as permission_types
  FROM system.information_schema.table_privileges
  WHERE grantee NOT LIKE '%admin%'
  GROUP BY grantee
)
SELECT
  username,
  table_access_count,
  permission_types,
  CASE 
    WHEN table_access_count > 20 THEN 'HIGH_RISK'
    WHEN table_access_count > 10 THEN 'MEDIUM_RISK'
    ELSE 'NORMAL'
  END as risk_level
FROM user_permission_count
WHERE table_access_count > 5
ORDER BY table_access_count DESC;

In [ ]:
-- Query 3: Identify stale accounts (no access in 90 days)

-- This would use audit logs:
-- SELECT DISTINCT username
-- FROM main.governance.access_audit_log
-- WHERE event_time < current_timestamp() - INTERVAL 90 DAYS
-- AND username NOT IN (
--   SELECT DISTINCT username
--   FROM main.governance.access_audit_log
--   WHERE event_time >= current_timestamp() - INTERVAL 90 DAYS
-- );

### Component 5: Disaster Recovery for Permissions

**Scenario:** You accidentally revoked all permissions! How do you recover?

**Task:** Create a backup system for grants.

In [ ]:
-- Backup all current grants

CREATE OR REPLACE TABLE main.governance.grants_backup AS
SELECT
  grantor,
  grantee,
  table_catalog,
  table_schema,
  table_name,
  privilege_type,
  is_grantable,
  current_timestamp() as backup_time
FROM system.information_schema.table_privileges;

In [ ]:
-- Create restore procedure (pseudo-code)

-- To restore grants from backup:
-- 1. Read from main.governance.grants_backup
-- 2. For each row, execute:
--    GRANT {privilege_type} ON TABLE {catalog}.{schema}.{table} TO `{grantee}`;

-- Example restore for one table:
SELECT 
  CONCAT(
    'GRANT ', privilege_type, 
    ' ON TABLE ', table_catalog, '.', table_schema, '.', table_name,
    ' TO `', grantee, '`;'
  ) as restore_statement
FROM main.governance.grants_backup
WHERE table_name = 'sensor_hourly'
  AND backup_time = (SELECT MAX(backup_time) FROM main.governance.grants_backup);

-- Copy and run these statements to restore!

### Component 6: Compliance Reporting Dashboard

**Task:** Create executive-friendly compliance reports.

In [ ]:
-- Compliance Report: Access Control Coverage

CREATE OR REPLACE VIEW main.governance.compliance_report AS
SELECT
  'Total Tables' as metric,
  COUNT(DISTINCT table_name) as value
FROM system.information_schema.tables
WHERE table_schema = 'e2eai_iot_turbine'

UNION ALL

SELECT
  'Tables with Access Controls' as metric,
  COUNT(DISTINCT table_name) as value
FROM system.information_schema.table_privileges
WHERE table_schema = 'e2eai_iot_turbine'

UNION ALL

SELECT
  'Users with Access' as metric,
  COUNT(DISTINCT grantee) as value
FROM system.information_schema.table_privileges
WHERE table_schema = 'e2eai_iot_turbine'
  AND grantee NOT LIKE '%admin%'

UNION ALL

SELECT
  'Sensitive Tables Protected' as metric,
  COUNT(*) as value
FROM system.information_schema.table_privileges
WHERE table_name IN ('turbine_maintenance', 'cost_analysis')
  AND privilege_type = 'SELECT'
  AND grantee IN ('executives', 'data_scientists');

In [ ]:
-- View the compliance report
SELECT * FROM main.governance.compliance_report;

### Component 7: Automated Alerts

**Task:** Set up alerts for security violations.

**Pseudo-code for alert system:**

```python
# Alert Rule 1: Unusual access patterns
if user_access_count > 1000 in last_hour:
    send_alert(f"User {username} accessed {table} {count} times")

# Alert Rule 2: Sensitive table access by unauthorized user
if accessed_table == 'turbine_cost_analysis' and user not in 'executives':
    send_critical_alert(f"Unauthorized access attempt by {username}")

# Alert Rule 3: Permission changes
if action in ['GRANT', 'REVOKE']:
    send_notification(f"Permission change: {action} {table} to {user}")
```

In Databricks, implement with:
- **Databricks SQL Alerts** on audit log queries
- **Webhooks** to Slack/Email/PagerDuty
- **Jobs** that run hourly/daily checks

## ✅ Validation Steps

### Governance Framework Checklist

1. **Documentation Complete**
   - [ ] Access control matrix filled out
   - [ ] Data classification defined
   - [ ] Incident response procedures documented
   - [ ] Compliance checklist created

2. **Technical Implementation**
   - [ ] Audit logging tables created
   - [ ] Lineage documented
   - [ ] Access review queries working
   - [ ] Grant backup system in place
   - [ ] Compliance reports generated

3. **Operational Readiness**
   - [ ] Team trained on playbook
   - [ ] Quarterly review schedule set
   - [ ] Alert system configured
   - [ ] Disaster recovery tested

### Test Your System

```sql
-- Test 1: Audit log captures access
SELECT * FROM main.governance.access_audit_log 
ORDER BY event_time DESC LIMIT 10;

-- Test 2: Compliance report shows 100% coverage
SELECT * FROM main.governance.compliance_report;

-- Test 3: Can restore grants from backup
-- (Run restore procedure)

-- Test 4: Excessive permissions detected
SELECT * FROM main.governance.excessive_permissions_report;
```

## 📊 Success Criteria

✅ Complete governance playbook documented  
✅ Audit logging system operational  
✅ Data lineage mapped  
✅ Automated access review queries created  
✅ Grant backup and restore system working  
✅ Compliance reporting dashboard built  
✅ Alert rules defined  

**Level 3 Complete!** 🏆 You're now a Governance Master!

---

# 🎓 Final Reflection & Next Steps

## 🎉 Congratulations!

You've built a **production-ready data governance framework** from scratch! 

### What You've Mastered

**Technical Skills:**
- ✅ Unity Catalog permission model (GRANT/REVOKE)
- ✅ Data masking and column-level security
- ✅ Row-level security with filtered views
- ✅ Audit logging and compliance reporting
- ✅ Data lineage analysis
- ✅ Disaster recovery procedures

**Governance Principles:**
- ✅ Principle of Least Privilege
- ✅ Defense in Depth (multiple security layers)
- ✅ Auditability and Transparency
- ✅ Separation of Duties
- ✅ Regular Access Reviews

---

## 🚀 Real-World Application

### How This Works in Production

**Scenario: New Employee Onboarding**
```
1. HR creates user account
2. Manager requests access via ticket
3. Governance team reviews playbook
4. Execute GRANT statements
5. User added to audit log
6. Confirmation email sent
```

**Scenario: Security Incident**
```
1. Alert triggered: Unusual access pattern
2. Security team reviews audit logs
3. Identify affected tables via lineage
4. REVOKE access immediately
5. Restore from grants backup if needed
6. Root cause analysis
```

**Scenario: Compliance Audit**
```
1. Auditor requests access reports
2. Run compliance dashboard queries
3. Export results to PDF
4. Demonstrate access controls
5. Show audit trail
6. Pass audit ✓
```

---

## 💡 Advanced Topics (For Later)

Once you've mastered the basics, explore:

### 1. **Attribute-Based Access Control (ABAC)**
Instead of role-based, use user attributes:
```sql
-- Grant access based on user location, department, clearance level
CREATE VIEW filtered_view AS
SELECT * FROM sensor_data
WHERE location = get_user_attribute('location')
  AND clearance_level >= get_user_attribute('clearance');
```

### 2. **Dynamic Data Masking**
Mask data based on context:
```sql
-- Show full data in production, masked in dev
CASE 
  WHEN current_database() = 'prod' THEN actual_value
  ELSE mask(actual_value)
END
```

### 3. **Time-Based Access**
Temporary permissions that auto-expire:
```sql
-- Grant access for 24 hours only
-- (Requires external automation)
```

### 4. **Cross-Catalog Governance**
Manage permissions across multiple catalogs:
```sql
GRANT SELECT ON main.* TO `analysts`;
GRANT SELECT ON dev.* TO `data_engineers`;
```

---

## 🎯 Next Steps in Your Journey

### Immediate Actions
1. **Apply to Your Data:** Use this framework on your actual datasets
2. **Share with Team:** Present your governance model to stakeholders
3. **Get Feedback:** What security concerns do they have?

### Continue Learning
- **Module 03 - BI Dashboards:** Create compliance dashboards
- **Module 04 - ML:** Governance for ML models and features
- **Module 05 - GenAI:** Security for AI agents accessing data

### Certification Prep
This challenge prepares you for:
- **Databricks Data Engineer Associate**
- **Databricks Data Analyst Associate**
- **Security/Compliance certifications**

---

## 📚 Resources

### Documentation
- [Unity Catalog Best Practices](https://docs.databricks.com/data-governance/unity-catalog/best-practices.html)
- [Data Governance Guide](https://docs.databricks.com/data-governance/index.html)
- [Audit Logs](https://docs.databricks.com/administration-guide/account-settings/audit-logs.html)

### Community
- Databricks Community Forums (Data Governance section)
- Unity Catalog GitHub discussions
- LinkedIn #DataGovernance

---

## 🏆 Challenge Badge Unlocked!

**🔐 Data Governance Master**

You've completed:
- ✅ Level 1: Access Control Designer
- ✅ Level 2: Security Architect  
- ✅ Level 3: Governance Master

**Share your achievement:**
> "Just completed a comprehensive data governance challenge covering Unity Catalog, data masking, audit logging, and compliance reporting! #DataGovernance #Databricks #UnityCatalog"

---

**Thank you for your dedication to secure, compliant data engineering!** 🎉

*Remember: Good governance isn't about blocking access - it's about enabling safe, responsible data use.*